# Day 4 (0725)

### 網頁開發應用
- Flask + local端應用
- 公共腳踏車地圖應用
- 空氣品質應用

### 雲端架設網站
- google GCE為例

### 資料庫
- sqlite3
- 收集空氣品質資料的排程 --> 為下星期做準備

### 網頁模版
- bootstrap
- 選一個樣版來試改

----

## Day4-1

### Demo1: 在local端，以Flask設計網頁程式

- Flask 是一個python的網頁微框架 (micro-framework) [link](https://flask.palletsprojects.com/en/1.1.x/)
- `pip install flask`
- 以下程式為一簡單的網頁功能：
    - / : 回傳 "OK"文字給 browser
    - /test : 回傳資料，並包裝成JSON格式
    - /day2-1 : 把即有的網頁傳送給browser顯示

### 在console下，下指令 `python app.py` 把web server跑起來
- 打開網址: http://localhost:5000/
- 打開網址: http://localhost:5000/test
- 打開網址: http://localhost:5000/day2-1



In [ ]:
import os
import random
import requests
import datetime, time
import csv

from functools import wraps
from flask import Flask, request, abort, render_template, Response
from flask import json, jsonify, session, redirect, url_for
from flask import send_file

VER = 'V2020.07.17'

#app = Flask(__name__)
app = Flask(__name__, static_url_path='', static_folder='static')

@app.route('/test')
def test():
    local_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    return jsonify({'result':'OK', 'localtime':local_time, 'version':VER})

@app.route('/day2-1')
def day2_1():
    return app.send_static_file('day2-1.html')

@app.route('/')
def index():
    return 'OK'

if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0')


## Demo2: 網頁伺服器：畫圖、傳送圖片

----
## 把程式放到 GCE 平台，架起自己的web server

- 把整個資料夾 zip 起來 (如：demo1_map.zip)
- 上傳 GCE ，會在家目錄 `~/` 看到這個檔案，使用指令 `unzip demo1_map.zip` 解壓縮
    - 如果是新開的vm，請先安裝pip3，指令： `sudo apt install python3-pip`
    - 如果指令有問題，請安裝unzip程式: `sudo apt install unzip`
    - 如果缺Flask套件，請下指令: `pip3 install flask`
- 同local端一樣的指令 
    - 先進到專案資料夾: `cd demo1_map`
    - 再執行程式: `python3 app.py`
- 打開google防火牆的port 5000 : 請參考 day4-1.pptx的說明
- 打開 http://your-ip:5000/ 就可以看到網頁

### 如果要讓網頁跑在一般的 80 port，請修改檔案`app.py`這一段為：

```
if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=80)
```

- 然後進GCE把防火牆的 http port (80 port)把打開
- 再執行指令: `sudo python3 app.py`


### *練習題：把自己的腳踏車地圖網頁加入

- 提示1: 產生的 bike.html 要上傳到GCE VM中，再用指令: `mv ~/bike.html ~/demo1_map/static`


In [1]:
import requests
import csv
import folium
import geocoder

url = "https://data.ntpc.gov.tw/api/datasets/71CD1490-A2DF-4198-BEF1-318479775E8A/csv/file"
r = requests.get(url)
decoded_content = r.content.decode('utf-8')
cr = csv.reader(decoded_content.splitlines(), delimiter=',')
data_list = list(cr)

location = geocoder.osm('新北市').latlng

m = folium.Map(location=location, zoom_start=14)

for item in data_list[1:]:
    try:
        name = item[1]
        area = item[4]
        total = item[2]
        n = item[3]
        lat = item[6]
        lng = item[7]
        if int(n)<5 and int(n)>0:
            folium.Marker([float(lat), float(lng)], popup=name+'<br>剩餘車輛:%s/%s' %(n,total),
                        icon=folium.Icon(color='red', prefix='fa', icon='fa-bicycle')).add_to(m)
        elif int(n)==0:
            folium.Marker([float(lat), float(lng)], popup=name+'<br>剩餘車輛:%s/%s' %(n,total),
                        icon=folium.Icon(color='black', prefix='fa', icon='fa-window-close')).add_to(m)
        else:
            folium.Marker([float(lat), float(lng)], popup=name+'<br>剩餘車輛:%s/%s' %(n,total),
                        icon=folium.Icon(color='green', prefix='fa', icon='fa-bicycle')).add_to(m)
        
    except Exception as e:
        print(e.args)
        
m.save('bike.html')

### *練習題：完成動態產生的空氣品質觀測資訊地圖網路